## import model from downloaded code

In [1]:
from artifacts.v76.gpcrscan.models.mamba_models import BestGPCR  # as model
import torch
from pathlib import Path

/home/lab09/.conda/envs/deply_aigpro/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# test data

In [2]:
# model = model.load_from_checkpoint(abs_checkpoint_Path)
abs_checkpoint_Path = "/home/lab09/DEV/Deployments/AiGPro/artifacts/model:v30/epoch=499-val_loss=0.59.ckpt"
model_ckpt = torch.load(abs_checkpoint_Path)
model_ckpt

{'epoch': 499,
 'global_step': 197500,
 'pytorch-lightning_version': '2.1.3',
 'state_dict': OrderedDict([('model.complete_conv.conv_layers.0.weight',
               tensor([[[ 0.4218, -0.5659, -0.5445]],
               
                       [[-0.0118,  0.3823,  0.1836]]], device='cuda:0')),
              ('model.complete_conv.conv_layers.3.weight',
               tensor([[[ 0.0230,  0.1707, -0.0083],
                        [ 0.3299,  0.0881, -0.2425]],
               
                       [[-0.0123, -0.0931, -0.2052],
                        [ 0.3929, -0.4043, -0.1675]],
               
                       [[ 0.2619,  0.1041, -0.3579],
                        [ 0.3489, -0.0147,  0.1476]],
               
                       [[ 0.3486,  0.0429,  0.0719],
                        [ 0.3625,  0.3916,  0.4064]],
               
                       [[-0.3530, -0.2560,  0.0466],
                        [-0.1001, -0.3054, -0.0121]],
               
                       [[ 0.394

# removel model. from the state_dict

In [5]:
for key in list(model_ckpt["state_dict"].keys()):
    if "model." in key:
        model_ckpt["state_dict"][key.replace("model.", "")] = model_ckpt[
            "state_dict"
        ].pop(key)

# Save the updated checkpoint
# updated_checkpoint_path = "updated_checkpoint.pth"
# torch.save(model_ckpt, updated_checkpoint_path)

In [8]:
# model = BestGPCR.load_from_checkpoint(abs_checkpoint_Path)

model = BestGPCR().to("cuda")
# model.load_from_checkpoint(updated_checkpoint_path)
model.load_state_dict(model_ckpt["state_dict"], strict=False)
model.eval()
model

None


BestGPCR(
  (complete_conv): Conv1DBlock(
    (conv_layers): Sequential(
      (0): Conv1d(1, 2, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): Conv1d(2, 8, kernel_size=(3,), stride=(1,), padding=(1,), dilation=(2,), bias=False)
      (4): ReLU(inplace=True)
      (5): Dropout(p=0.1, inplace=False)
      (6): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=(1,), dilation=(4,), bias=False)
      (7): ReLU(inplace=True)
      (8): Dropout(p=0.1, inplace=False)
      (9): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,), dilation=(8,), bias=False)
      (10): ReLU(inplace=True)
      (11): Dropout(p=0.1, inplace=False)
      (12): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (13): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14): AdaptiveMaxPool1d(output_size=32)
    )
  )
  (complete_conv1): Conv1DBlock(
 

In [9]:
# protein, ligand, smi_desc, charge_fp, clabel, emb
batch = 128
protein = torch.ones(batch, 1900).long()
ligand = torch.ones(batch, 1, 100).long()
smi_desc = torch.randn(batch, 1, 170)
charge_fp = torch.randn(batch, 1, 1024)
clabel = torch.ones(batch).view(-1).long()
emb = torch.randn(batch, 1, 1024)

# shapes

test_data = (protein, ligand, smi_desc, charge_fp, clabel, emb)
protein.shape, ligand.shape, smi_desc.shape, charge_fp.shape, clabel.shape, emb.shape

(torch.Size([128, 1900]),
 torch.Size([128, 1, 100]),
 torch.Size([128, 1, 170]),
 torch.Size([128, 1, 1024]),
 torch.Size([128]),
 torch.Size([128, 1, 1024]))

In [9]:
from pathlib import Path
import lightning as L
import torch

model.eval()
save_model_name = "model.pt"

with torch.no_grad():
    # move to cuda
    model = model.cuda()
    test_data = [x.cuda() for x in test_data]
    y_hat = model(test_data)
    print(f"y_hat: {y_hat}")
    # save to torchscript
    try:
        torch.jit.save(model.to_torchscript(), save_model_name)
        print(
            f"Saved torchscript model to {Path(save_model_name).resolve().absolute()}"
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Trying to trace the model")
        model._trainer = L.Trainer()
        trace_script = torch.jit.trace(model, (test_data,))
        trace_script.save(save_model_name)
        # torch.jit.save(trace_script, save_model_name)
        print(f"Saved traced model to {Path(save_model_name).resolve().absolute()}")

y_hat: tensor([[6.9837],
        [7.0731],
        [7.1442],
        [6.8591],
        [6.4491],
        [7.0730],
        [6.8307],
        [7.2882],
        [6.4409],
        [6.7318],
        [7.0341],
        [7.0335],
        [6.5779],
        [6.8652],
        [7.1741],
        [7.1128],
        [6.7449],
        [7.1599],
        [6.9255],
        [7.0011],
        [7.0464],
        [6.9378],
        [6.5064],
        [6.7671],
        [7.2035],
        [6.5766],
        [7.0813],
        [7.1410],
        [6.7108],
        [7.0992],
        [7.1921],
        [7.0468],
        [7.0413],
        [6.9745],
        [7.1275],
        [6.5766],
        [7.0343],
        [7.2289],
        [6.9958],
        [6.6727],
        [7.2671],
        [6.6669],
        [6.8936],
        [6.8134],
        [7.2331],
        [7.0715],
        [7.0823],
        [7.1112],
        [7.0216],
        [6.8994],
        [7.0159],
        [6.4497],
        [6.6288],
        [7.6169],
        [7.0966],
   

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Error: Unsupported value kind: Tensor
Trying to trace the model


/home/lab09/DEV/Deployments/AiGPro/artifacts/v76/gpcrscan/models/component.py:172: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.cached_penc is not None and self.cached_penc.shape == tensor.shape:


Saved traced model to /home/lab09/DEV/Deployments/AiGPro/model.pt


In [10]:
# Load

loaded_model = torch.jit.load("model.pt")

In [11]:
loaded_model(test_data)

tensor([[6.9837],
        [7.0731],
        [7.1442],
        [6.8591],
        [6.4491],
        [7.0730],
        [6.8307],
        [7.2882],
        [6.4409],
        [6.7318],
        [7.0341],
        [7.0335],
        [6.5779],
        [6.8652],
        [7.1741],
        [7.1128],
        [6.7449],
        [7.1599],
        [6.9255],
        [7.0011],
        [7.0464],
        [6.9378],
        [6.5064],
        [6.7671],
        [7.2035],
        [6.5766],
        [7.0813],
        [7.1410],
        [6.7108],
        [7.0992],
        [7.1921],
        [7.0468],
        [7.0413],
        [6.9745],
        [7.1275],
        [6.5766],
        [7.0343],
        [7.2289],
        [6.9958],
        [6.6727],
        [7.2671],
        [6.6669],
        [6.8936],
        [6.8134],
        [7.2331],
        [7.0715],
        [7.0823],
        [7.1112],
        [7.0216],
        [6.8994],
        [7.0159],
        [6.4497],
        [6.6288],
        [7.6169],
        [7.0966],
        [6

## only predict

In [12]:
model.eval()
with torch.no_grad():
    # move to cuda
    model = model.cuda()
    test_data = [x.cuda() for x in test_data]
    y_hat = model(test_data)
    print(f"y_hat: {y_hat}")

y_hat: tensor([[6.9837],
        [7.0731],
        [7.1442],
        [6.8591],
        [6.4491],
        [7.0730],
        [6.8307],
        [7.2882],
        [6.4409],
        [6.7318],
        [7.0341],
        [7.0335],
        [6.5779],
        [6.8652],
        [7.1741],
        [7.1128],
        [6.7449],
        [7.1599],
        [6.9255],
        [7.0011],
        [7.0464],
        [6.9378],
        [6.5064],
        [6.7671],
        [7.2035],
        [6.5766],
        [7.0813],
        [7.1410],
        [6.7108],
        [7.0992],
        [7.1921],
        [7.0468],
        [7.0413],
        [6.9745],
        [7.1275],
        [6.5766],
        [7.0343],
        [7.2289],
        [6.9958],
        [6.6727],
        [7.2671],
        [6.6669],
        [6.8936],
        [6.8134],
        [7.2331],
        [7.0715],
        [7.0823],
        [7.1112],
        [7.0216],
        [6.8994],
        [7.0159],
        [6.4497],
        [6.6288],
        [7.6169],
        [7.0966],
   

# Predict


In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
import numpy as np
import torch
from gpcrscan.core.pdbdataset import GPCRDatasetCombined
from gpcrscan.chem.descriptors import morgan_fingerprint

SMILES_VOCAB loaded of len :  575


/home/lab09/.conda/envs/deply_aigpro/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CHemcal int

In [12]:
# msa_int, chem_int, smile_desc, charge_morgan, label
from artifacts.v76.gpcrscan.utils.utilities import get_pad_array
from artifacts.v76.gpcrscan.chem.old_tokenizer import tokenize_smiles

canonical_smile = "CC(C)C1=CC(=CC=C1)C(=O)NC2=CC(=C(C=C2)O)C3=CC=CC=C3"
print(tokenize_smiles(canonical_smile))
chem_int = torch.Tensor(tokenize_smiles(canonical_smile)).unsqueeze(0)
chem_int = get_pad_array(chem_int, (1, 100), -1)
chem_int = torch.tensor(chem_int).long()
chem_int

[39, 39, 4, 39, 5, 39, 6, 35, 39, 39, 35, 39, 4, 39, 35, 39, 6, 5, 39, 4, 35, 41, 5, 40, 39, 6, 35, 39, 39, 35, 39, 4, 39, 35, 39, 6, 5, 39, 6, 35, 39, 39, 35, 39, 39, 35, 39, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


tensor([[39, 39,  4, 39,  5, 39,  6, 35, 39, 39, 35, 39,  4, 39, 35, 39,  6,  5,
         39,  4, 35, 41,  5, 40, 39,  6, 35, 39, 39, 35, 39,  4, 39, 35, 39,  6,
          5, 39,  6, 35, 39, 39, 35, 39, 39, 35, 39,  6,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

# MSA int

In [26]:
# geterate final unique Uniprot ids and MSA
from artifacts.v76.gpcrscan.utils.utilities import load_dataframe
import pandas as pd

train_data = "/home/lab09/DEV/GPCRScan/data/AzothBio/v5/v7_train.parquet"
test_data = (
    "/home/lab09/DEV/GPCRScan/data/AzothBio/v5/v6_less_interaction_proteins.parquet"
)


traindf = load_dataframe(train_data)
testdf = load_dataframe(test_data)
traindf = pd.concat([traindf, testdf])
traindf.head()

,combined_id,standard_value,uniprot_id,inchi_key,endpoint,dataset,standard_units,protein_seq,canonical_smiles,smiles,...,charge_fp,selfies,exclude_terminal,entry_name,label,morgan_fingerprint,plog_endpoint,mw,logp,improve_iden
0,P35348|ZVZCHVBOZRWHPS-UHFFFAOYSA-N,1.2200,P35348,ZVZCHVBOZRWHPS-UHFFFAOYSA-N,0.985607,train,nM,----------------------------------------------...,COc1cc(N2C(=O)c3ccc(C(=O)NCC(O)CN4CCN(c5ccccc5...,COc1cc(N2C(=O)c3ccc(C(=O)NCC(O)CN4CCN(c5ccccc5...,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[C][O][C][=C][C][Branch2][Branch1][#Branch2][N...,-------PVNISKAILLGVILGG-LILFGVLGNILVILSVACH---...,ada1a_human,0,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.913640,632.284614,3.21310,P35348|ZVZCHVBOZRWHPS-UHFFFAOYSA-N|0
1,P25101|SYCPGPBCEDENBL-UHFFFAOYSA-N,85.0000,P25101,SYCPGPBCEDENBL-UHFFFAOYSA-N,0.678430,train,nM,----------------------------------------------...,COc1cccc(NC(=O)c2sccc2S(=O)(=O)Nc2onc(C)c2Br)c1,COc1cccc(NC(=O)c2sccc2S(=O)(=O)Nc2onc(C)c2Br)c1,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[C][O][C][=C][C][=C][C][Branch2][Ring1][S][N][...,---PQQTKITSAFKYINTVISCT-IFIVGMVGNATLLRIIYQN---...,ednra_human,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",7.070581,470.955825,3.86872,P25101|SYCPGPBCEDENBL-UHFFFAOYSA-N|0
2,Q03431|JWIXYUKPAMOBSU-UHFFFAOYSA-N,50118.6992,Q03431,JWIXYUKPAMOBSU-UHFFFAOYSA-N,0.216667,train,nM,----------------------------------------------...,O=C(NC(=S)Nc1cccc([N+](=O)[O-])c1)C1CCCC1,O=C(NC(=S)Nc1cccc([N+](=O)[O-])c1)C1CCCC1,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",[O][=C][Branch2][Ring1][=Branch1][N][C][=Branc...,--FLTNETREREVFDRLGMIYTV-GYSVSLASLTVAVLILAYF---...,pth1r_human,1,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",4.300000,293.083412,2.59800,Q03431|JWIXYUKPAMOBSU-UHFFFAOYSA-N|1
3,P41145|PHEIVMXYEVVNHK-UHFFFAOYSA-N,1014.0000,P41145,PHEIVMXYEVVNHK-UHFFFAOYSA-N,0.498994,train,nM,----------------------------------------------...,O=C1N(CCNC2CC2)CN(c2ccccc2)C12CCN(C(c1ccccc1Cl...,O=C1N(CCNC2CC2)CN(c2ccccc2)C12CCN(C(c1ccccc1Cl...,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",[O][=C][N][Branch1][=Branch2][C][C][N][C][C][C...,-----------AIPVIITAVYSV-VFVVGLVGNSLVMFVIIRY---...,oprk_human,0,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5.993962,548.210967,5.97570,P41145|PHEIVMXYEVVNHK-UHFFFAOYSA-N|0
4,P21728|PGWPCIPGEIHMJD-UHFFFAOYSA-N,12325.1885,P21728,PGWPCIPGEIHMJD-UHFFFAOYSA-N,0.318201,train,nM,----------------------------------------------...,O=C(CCCC(O)c1ccccc1)NCCC1=CCc2ccccc21,O=C(CCCC(O)c1ccccc1)NCCC1=CCc2ccccc21,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",[O][=C][Branch1][S][C][C][C][C][Branch1][C][O]...,-----------SVRILTACFLSL-LILSTLLGNTLVCAAVIRF---...,drd1_human,0,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.909206,335.188529,4.03630,P21728|PGWPCIPGEIHMJD-UHFFFAOYSA-N|0


In [27]:
msa_df = traindf[["uniprot_id", "protein_seq"]]
msa_df = msa_df.drop_duplicates()
msa_df.head()

,uniprot_id,protein_seq
0,P35348,----------------------------------------------...
1,P25101,----------------------------------------------...
2,Q03431,----------------------------------------------...
3,P41145,----------------------------------------------...
4,P21728,----------------------------------------------...


In [28]:
msa_df.shape

(231, 2)

In [29]:
msa_df.to_parquet("MSA_DF.parquet", index=False)

In [33]:
def get_msa_for_uniprot(uniprot_id):
    # get msa
    msa = load_dataframe("MSA_DF.parquet")
    msa = msa[msa["uniprot_id"] == uniprot_id]
    return msa.protein_seq.values[0]


msa = get_msa_for_uniprot("P35348")
msa

'--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------MVFLSGNASDSSNCTQPPA-------PVNISKAILLGVILGG-LILFGVLGNILVILSVACH--------------RHLH--------------SVTHYYIVNLAVADLLLTSTVLPFSAIFEVLG--------------------YW-AF-----------------GRVFCNIWAAVDVLCCTASIMGLCIISIDRYIGVSY----------------------PLRYPTIV-----------------TQRRGLMALLCV-WALSLVISIGPLFGW-----RQPAPEDETI-----------------------------------------------CQI-----------------------------------------------------------------

In [35]:
from artifacts.v76.gpcrscan.chem.tokenizers import load_custom_tokenizer

gpcr_protein_tokenizer_filename = "GPCR_prot_tokenizer.json"

ProteinTokenizer = load_custom_tokenizer(
    gpcr_protein_tokenizer_filename,
)

In [36]:
msa_int = ProteinTokenizer(
    msa, max_length=1900, padding="max_length", truncation=True, return_tensors="pt"
).input_ids[
    0
]  # type: ignore
msa_int

tensor([4, 4, 4,  ..., 2, 2, 2])

# MORGAN fp

In [183]:
from concurrent.futures import ThreadPoolExecutor
from typing import List
import pandas as pd
from copy import deepcopy
import swifter
from artifacts.v76.gpcrscan.chem.tokenizers import load_custom_tokenizer
from artifacts.v76.gpcrscan.utils.utilities import load_dataframe

from artifacts.v76.gpcrscan.utils.utilities import get_pad_array
from artifacts.v76.gpcrscan.chem.old_tokenizer import tokenize_smiles
from artifacts.v76.gpcrscan.chem.descriptors import (
    compute_ligand_descriptors,
    generate_charge_fingerprint,
    morgan_fingerprint,
)


def canonical_smile(smi):
    try:
        mol = Chem.MolFromSmiles(smi, sanitize=True)
        return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=True)
    except:
        return smi


def charge_fp(smi):
    try:
        return list(generate_charge_fingerprint(smi))
    except:
        return None


def compute_desc(smi):
    try:
        return compute_ligand_descriptors(smi)
    except:
        return None


class DataPrep:
    def __init__(
        self,
        smi: List[str] | str = None,
        gpcr_protein_tokenizer_filename=None,
        msa_filename=None,
        canonical_smile_col="canonical_smiles",
        uniprot_id_col="uniprot_id",
        global_scan: bool = False,
    ) -> None:

        if isinstance(smi, list):
            self.smi_df = pd.DataFrame({"smiles": smi})
        else:
            self.smi_df = load_dataframe(smi)
        self.global_scan = global_scan
        # print(canonical_smile(smi[0]))

        if gpcr_protein_tokenizer_filename is not None:
            self.gpcr_protein_tokenizer_filename = gpcr_protein_tokenizer_filename

        else:
            self.gpcr_protein_tokenizer_filename = "GPCR_prot_tokenizer.json"

        if msa_filename is None:
            msa_filename = "MSA_DF.parquet"
            self.msa_df = load_dataframe(msa_filename)
        else:
            self.msa_df = load_dataframe(msa_filename)

        self.canonical_smile_col = canonical_smile_col
        self.uniprot_id_col = uniprot_id_col
        # print(self.msa_df.head())

        self.ProteinTokenizer = load_custom_tokenizer(
            self.gpcr_protein_tokenizer_filename,
        )
        self.__add_data()

    def __add_data(self):

        #  with ThreadPoolExecutor() as executor:
        #     futures = [
        #         executor.submit(self.add_msa_token),
        #         executor.submit(self.get_canonical_smiles),
        #         executor.submit(self.add_mfp),
        #         executor.submit(self.add_chemical_token),
        #         executor.submit(self.get_charge_fp),
        #         executor.submit(self.get_desc),
        #         executor.submit(self.add_msa)
        #     ]

        #     # Wait for all tasks to complete
        #     for future in futures:
        #         future.result()

        self.add_msa_token()
        self.get_canonical_smiles()
        self.add_mfp()
        self.add_chemical_token()
        self.get_charge_fp()
        self.get_desc()
        self.add_msa()

    def get_msa_token(self, uniprot_id):

        msa = self.msa_df[
            self.msa_df[self.uniprot_id_col] == uniprot_id
        ].protein_seq.values[0]
        msa_int = self.ProteinTokenizer(
            msa,
            max_length=1900,
            padding="max_length",
            truncation=True,
            return_tensors="np",
        ).input_ids[0]
        return msa_int

    def add_msa_token(self):
        self.msa_df["msa_int"] = self.msa_df.swifter.allow_dask_on_strings(
            enable=True
        ).apply(lambda x: self.get_msa_token(x[self.uniprot_id_col]), axis=1)

    def get_chem_token(self, canonical_smile):
        chem_int = torch.Tensor(tokenize_smiles(canonical_smile)).unsqueeze(0)
        chem_int = get_pad_array(chem_int, (1, 100), -1)
        # chem_int = torch.tensor(chem_int).long()
        return chem_int

    def add_chemical_token(self):
        self.smi_df["chem_int"] = self.smi_df.swifter.allow_dask_on_strings(
            enable=True
        ).apply(lambda x: tokenize_smiles(x[self.canonical_smile_col]), axis=1)

    @property
    def df(self):
        return self.smi_df

    def get_canonical_smiles(self):

        if self.canonical_smile_col in self.smi_df.columns:
            self.smi_df = self.smi_df.drop(columns=[self.canonical_smile_col])

        self.smi_df[self.canonical_smile_col] = (
            self.smi_df.swifter.allow_dask_on_strings(enable=True).apply(
                lambda x: canonical_smile(x["smiles"]), axis=1
            )
        )

    def get_charge_fp(self):
        self.smi_df["charge_fp"] = self.smi_df.swifter.allow_dask_on_strings(
            enable=True
        ).apply(lambda x: charge_fp(x[self.canonical_smile_col]), axis=1)

    def get_desc(self):
        self.smi_df["desc"] = self.smi_df.swifter.allow_dask_on_strings(
            enable=True
        ).apply(lambda x: compute_desc(x[self.canonical_smile_col]), axis=1)

    def add_msa(self):
        _all_df = pd.DataFrame()
        _global_df = self.smi_df.copy()
        if self.global_scan:
            for smi in _global_df[self.canonical_smile_col]:
                _tmp_df = _global_df[_global_df[self.canonical_smile_col] == smi]
                _tmp_df = _tmp_df.loc[
                    _tmp_df.index.repeat(len(self.msa_df))
                ].reset_index(drop=True)
                _tmp_df = pd.concat([_tmp_df, self.msa_df.copy()], axis=1, join="outer")
                _all_df = pd.concat([_all_df, _tmp_df], axis=0)
        else:
            _all_df = _global_df.copy()
            assert (
                "uniprot_id" in _all_df.columns
            ), "uniprot_id not in the input dataframe"
            ## add msa row for matching uniprot_id
            _all_df = pd.merge(
                _all_df,
                self.msa_df,
                left_on="uniprot_id",
                right_on="uniprot_id",
                how="left",
            )

            # _all_df = pd.concat([_all_df, self.msa_df.copy()], axis=1, join="outer")
        if "label" in _all_df.columns:
            _all_df = _all_df.rename(columns={"label": "original_label"})

        _all_df["label"] = 1
        _all_df_2 = deepcopy(_all_df)
        _all_df_2["label"] = 0
        _all_df = pd.concat([_all_df, _all_df_2], axis=0)

        self.smi_df = _all_df.reset_index(drop=True)
        # self.smi_df = _all_df.reset_index(drop=True)
        # self.smi_df["Unit"] = ["nM"]
        ## add Standard Value

    def add_mfp(self):
        self.smi_df["mfp"] = self.smi_df.swifter.allow_dask_on_strings(
            enable=True
        ).apply(lambda x: morgan_fingerprint(x[self.canonical_smile_col]), axis=1)

    @staticmethod
    def canonical_smile(smi):
        try:
            mol = Chem.MolFromSmiles(smi, sanitize=True)
            return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=True)
        except Exception as e:
            print(e)
            return smi

    @staticmethod
    def charge_fp(smi):
        try:
            return list(generate_charge_fingerprint(smi))
        except Exception as e:
            print(e)
            return None

    @staticmethod
    def compute_desc(smi):
        try:
            return compute_ligand_descriptors(smi)
        except Exception as e:
            print(e)
            return None

    @staticmethod
    def compute_mfp(smi):
        try:
            return list(morgan_fingerprint(smi))
        except Exception as e:
            print(e)
            return None


prep_data = DataPrep(
    [
        "CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1",
        "CCCN(CCC)[C@H]1CC[C@H](Nc2nc(NCc3ccc(-c4ccccc4)cc3)c3ncn(C(C)C)c3n2)CC1",
        "O=C(O)c1ccccc1C(=O)Oc1cccc(CO[N+](=O)[O-])c1",
        "[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(Nc4nccc(-c5cccnc5)n4)c3)cc2)CC1",
    ],
    global_scan=True,
)
prep_data.df

Pandas Apply:   0%|          | 0/231 [00:00<?, ?it/s]

Pandas Apply: 100%|██████████| 4/4 [00:00<00:00, 109.89it/s]


,smiles,canonical_smiles,mfp,chem_int,charge_fp,desc,uniprot_id,protein_seq,msa_int,label
0,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39, 39, 4, 39, 5, 39, 6, 35, 39, 39, 35, 39, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.33903741860993, -0.08540070966658253, 12.3...",P30874,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",1
1,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39, 39, 4, 39, 5, 39, 6, 35, 39, 39, 35, 39, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.33903741860993, -0.08540070966658253, 12.3...",P07550,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",1
2,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39, 39, 4, 39, 5, 39, 6, 35, 39, 39, 35, 39, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.33903741860993, -0.08540070966658253, 12.3...",Q8IYL9,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",1
3,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39, 39, 4, 39, 5, 39, 6, 35, 39, 39, 35, 39, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.33903741860993, -0.08540070966658253, 12.3...",P35346,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",1
4,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39, 39, 4, 39, 5, 39, 6, 35, 39, 39, 35, 39, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.33903741860993, -0.08540070966658253, 12.3...",Q8WXD0,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",1
...,...,...,...,...,...,...,...,...,...,...
1843,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[146, 39, 4, 146, 5, 4, 146, 5, 40, 6, 39, 39,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.970218546030445, -2.033217097618535, 12.97...",Q8TDS4,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",0
1844,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[146, 39, 4, 146, 5, 4, 146, 5, 40, 6, 39, 39,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.970218546030445, -2.033217097618535, 12.97...",P28336,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",0
1845,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[146, 39, 4, 146, 5, 4, 146, 5, 40, 6, 39, 39,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.970218546030445, -2.033217097618535, 12.97...",O95977,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",0
1846,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[146, 39, 4, 146, 5, 4, 146, 5, 40, 6, 39, 39,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(12.970218546030445, -2.033217097618535, 12.97...",Q15391,----------------------------------------------...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",0


In [184]:
prep_data.df.canonical_smiles.value_counts()

canonical_smiles
CC(C)C1=CC=C(C=C1)C(=O)NC1=CC=C(C=C1)C1=CC=CC=C1                                 462
CCCN(CCC)[C@H]1CC[C@H](Nc2nc(NCc3ccc(-c4ccccc4)cc3)c3ncn(C(C)C)c3n2)CC1          462
O=C(O)c1ccccc1C(=O)Oc1cccc(CO[N+](=O)[O-])c1                                     462
[2H]C([2H])([2H])N1CCN(Cc2ccc(C(=O)Nc3ccc(C)c(Nc4nccc(-c5cccnc5)n4)c3)cc2)CC1    462
Name: count, dtype: int64

# DataLoader

In [185]:
class PredictDataloader(torch.utils.data.Dataset):
    def __init__(self, data: pd.DataFrame, device="cuda"):
        self.data = data
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.device = device

        columns = ["msa_int", "chem_int", "desc", "charge_fp", "label", "mfp"]
        assert all(
            [x in self.data.columns for x in columns]
        ), f"columns not found: {columns}"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        protein = torch.tensor(row.msa_int).long().to(self.device)
        ligand = torch.tensor(row.chem_int).long().to(self.device)
        smi_desc = torch.tensor(row.desc).float().to(self.device)
        charge_fp = torch.tensor(row.charge_fp).float().to(self.device).unsqueeze(0)
        clabel = torch.tensor(row.label).long().to(self.device)
        mfp = torch.tensor(row.mfp).float().to(self.device).unsqueeze(0)
        # shape

        charge_morgan = torch.cat([charge_fp, mfp], dim=-1)
        # print(protein.shape, ligand.shape, smi_desc.shape, charge_morgan.shape)

        return protein, ligand, smi_desc, charge_morgan, clabel, clabel


dataloaders = PredictDataloader(prep_data.df)
for i in range(len(dataloaders)):
    print(dataloaders[i])
    break

(tensor([4, 4, 4,  ..., 2, 2, 2], device='cuda:0'), tensor([39, 39,  4, 39,  5, 39,  6, 35, 39, 39, 35, 39,  4, 39, 35, 39,  6,  5,
        39,  4, 35, 41,  5, 40, 39,  6, 35, 39, 39, 35, 39,  4, 39, 35, 39,  6,
         5, 39,  6, 35, 39, 39, 35, 39, 39, 35, 39,  6,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0], device='cuda:0'), tensor([ 1.2339e+01, -8.5401e-02,  1.2339e+01,  8.5401e-02,  6.5427e-01,
         3.1542e+02,  2.9425e+02,  3.1516e+02,  1.2000e+02,  7.5000e-01,
         1.3333e+00,  1.9167e+00,  1.7025e+00,  8.0285e+02,  1.6941e+01,
         1.3991e+01,  1.3991e+01,  1.1631e+01,  8.2009e+00,  8.2009e+00,
         6.1742e+00,  6.1742e+00,  4.0894e+00,  4.0894e+00,  2.5993e+00,
         2.5993e+00, -2.8700e+00,  1.6004e+01,  7.1360e+00,  3.7170e+00,
         1.4272e+02,  0.0000e+00,  4.7945e+

# Prediction

## converted script

In [186]:
from copy import deepcopy
import numpy as np
from dataclasses import dataclass

@dataclass
class PredResult:
    agonist: pd.DataFrame
    antagonist: pd.DataFrame
    combined: pd.DataFrame
    raw: pd.DataFrame

    def __repr__(self):
        return f"Agonist: {self.agonist.shape}, Antagonist: {self.antagonist.shape}, Combined: {self.combined.shape}, Raw: {self.raw.shape}"


class AiGProScript:
    def __init__(self, script_file, device="cuda", batch_size=128):
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.device = device
        self.model = torch.jit.load(script_file)
        self.model.eval()
        self.model = self.model.to(self.device)
        self.batch_size = batch_size

    @torch.no_grad()
    def predict(self, test_data):
        loaded_data = self.data_loader(test_data)
        # test_data = [x.to(self.device) for x in test_data]

        predictions = []
        for i in loaded_data:
            pred = self.model(i)
            predictions.append(pred.cpu().numpy().flatten())

        return np.concatenate(predictions).flatten()
        # y_hat = self.model(loaded_data)
        return y_hat.flatten().cpu().numpy()

    def data_loader(self, data_frame):
        return torch.utils.data.DataLoader(
            PredictDataloader(data_frame, device=None),
            batch_size=self.batch_size,
            shuffle=False,
        )

    def df_predict(self, data_frame):
        predictions = self.predict(data_frame)
        data_frame["pred"] = predictions

        _raw = deepcopy(data_frame.copy())

        agonist = data_frame[data_frame["label"] == 1].reset_index(drop=True)
        agonist["label"] = [1] * agonist.shape[0]
        agonist = agonist.rename(columns={"pred": "ago_pred"})
        # agonist = agonist.drop(columns=["label"])

        antagonist = data_frame[data_frame["label"] == 0].reset_index(drop=True)
        antagonist["label"] = [0] * antagonist.shape[0]
        antagonist = antagonist.rename(columns={"pred": "anta_pred"})
        # antagonist = antagonist.drop(columns=["label"])
        drop_cols = [
            "msa_int",
            "chem_int",
            "desc",
            "charge_fp",
            "mfp",
            "protein_seq",
            # "canonical_smiles",
        ]
        # remove _x and _y
        for col in drop_cols:
            if col in agonist.columns:
                agonist = agonist.drop(columns=[col])
            if col in antagonist.columns:
                antagonist = antagonist.drop(columns=[col])
        combined = pd.merge(
            agonist, antagonist, on=["canonical_smiles", "uniprot_id"], how="outer"
        )

        # remove all _y and rename _x to original
        for col in combined.columns:
            if col.endswith("_x"):
                combined = combined.rename(columns={col: col.replace("_x", "")})
            if col.endswith("_y"):
                combined = combined.drop(columns=[col])
        combined = combined.drop(columns=["label"])


        return PredResult(
            agonist,
            antagonist,
            combined.reset_index(drop=True),
            _raw,
        )

## predict

In [187]:
AiGPro = AiGProScript("model.pt")
# y_hat = AiGPro.predict(prep_data.df)
y_hat_df = AiGPro.df_predict(prep_data.df)
y_hat_df

Agonist: (924, 5), Antagonist: (924, 5), Combined: (924, 5), Raw: (1848, 11)

# Predictions Metrics

In [188]:
from artifacts.v76.gpcrscan.core import metrics

In [189]:
test_filename = "/home/lab09/DEV/GPCRScan/data/AzothBio/v5/v5_combined_test.parquet"
# test_df = load_dataframe(test_filename)
# test_df.head()

In [190]:
test_df = DataPrep(test_filename).df

Pandas Apply: 100%|██████████| 231/231 [00:00<00:00, 314.29it/s]


SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575


Pandas Apply: 100%|██████████| 11464/11464 [00:00<00:00, 58516.46it/s]


SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575


Pandas Apply: 100%|██████████| 11464/11464 [00:07<00:00, 1585.47it/s]


SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575
SMILES_VOCAB loaded of len :  575


Pandas Apply: 100%|██████████| 11464/11464 [01:31<00:00, 125.16it/s]


## Gard to check NaN-> Unsupported GPCRS

In [191]:
# Nan if OOD GPCRS, manually create token if not in the df MSA_DF.parquet | TODO
count = 0
for i in range(len(test_df)):
    try:
        assert len(test_df.iloc[i]["msa_int"]) == 1900
    # print(len(test_df.iloc[i]["msa_int"])
    except Exception as e:
        print(e, "<<<<<<<<<<<<<<<<<<<<<<<<<<<")
        count += 1
        print(test_df.iloc[i]["msa_int"])
    # break

print(f"Total count: {count}")
assert (
    count == 0
), "Failed on Some Porteins, maybe it is unsuppoted GPCR"  # add  but give warinings for unsupported GPCR

Total count: 0


In [192]:
# drop nan msa_int
clean_test_df = test_df.dropna(subset=["msa_int"])

In [193]:
test_pred_df = AiGPro.df_predict(clean_test_df)

In [194]:
test_pred_df

Agonist: (11464, 21), Antagonist: (11464, 21), Combined: (11480, 21), Raw: (22928, 26)

In [195]:
# label and  standard_label
absolute_data = test_pred_df.raw[
    test_pred_df.raw["original_label"] == test_pred_df.raw["label"]
]
absolute_data.shape

(11464, 26)

# Combined predict

In [196]:
new_metrics = metrics.metric(absolute_data.plog_endpoint, absolute_data.pred)

───────────────────────────────────────────────── Metric Results ──────────────────────────────────────────────────

R2: 0.833

R2M: 0.834

MSE: 0.345

R: 0.913

...

MAE: 0.397

RMSE: 0.587

Q2: 0.833

CI: 0.848

─────────────────────────────────────────────────────── END ───────────────────────────────────────────────────────

In [197]:
test_pred_df.antagonist.shape, test_pred_df.raw[
    test_pred_df.raw["original_label"] == 0
].shape, test_pred_df

((11464, 21),
 (10190, 26),
 Agonist: (11464, 21), Antagonist: (11464, 21), Combined: (11480, 21), Raw: (22928, 26))

# individua; Type

In [198]:
# new_metrics = metrics.metric(absolute_data.plog_endpoint, absolute_data.pred)
new_metrics1 = metrics.metric(
    test_pred_df.raw[
        (test_pred_df.raw["original_label"] == 0) & (test_pred_df.raw["label"] == 0)
    ].plog_endpoint,
    test_pred_df.raw[
        (test_pred_df.raw["original_label"] == 0) & (test_pred_df.raw["label"] == 0)
    ].pred,
)

───────────────────────────────────────────────── Metric Results ──────────────────────────────────────────────────

R2: 0.797

R2M: 0.798

MSE: 0.405

R: 0.893

...

MAE: 0.438

RMSE: 0.637

Q2: 0.797

CI: 0.864

─────────────────────────────────────────────────────── END ───────────────────────────────────────────────────────

In [199]:
new_metrics2 = metrics.metric(
    test_pred_df.raw[
        (test_pred_df.raw["original_label"] == 1) & (test_pred_df.raw["label"] == 1)
    ].plog_endpoint,
    test_pred_df.raw[
        (test_pred_df.raw["original_label"] == 1) & (test_pred_df.raw["label"] == 1)
    ].pred,
)

───────────────────────────────────────────────── Metric Results ──────────────────────────────────────────────────

R2: 0.724

R2M: 0.727

MSE: 0.296

R: 0.853

...

MAE: 0.363

RMSE: 0.544

Q2: 0.724

CI: 0.737

─────────────────────────────────────────────────────── END ───────────────────────────────────────────────────────